In [1]:
import pandas as pd
import numpy as np
import string
import nltk

In [2]:
df = pd.read_csv("final_data.csv")

In [3]:
df.head()

,Season,Episode,Transcript,Rating
0,1,1,"Scene: Central Perk, Chandler, Joey, Phoebe, a...","8.2 (8,746)"
1,1,2,"Scene Central Perk, everyone's there.]\nMonica...","7.9 (6,745)"
2,1,3,"Scene: Central Perk, everyone but Phoebe is th...","8.0 (6,346)"
3,1,4,"Scene: Central Perk, everyone is there except ...","8.0 (6,162)"
4,1,5,"Scene: Central Perk, all six are there.]\nMoni...","8.4 (6,153)"


In [4]:
splitter = lambda x: x.split("(")[0]
df[['Rating']] = df.Rating.astype('string').apply(splitter)

In [5]:
df.Rating = df.Rating.astype('float')

In [6]:
df.head()

,Season,Episode,Transcript,Rating
0,1,1,"Scene: Central Perk, Chandler, Joey, Phoebe, a...",8.2
1,1,2,"Scene Central Perk, everyone's there.]\nMonica...",7.9
2,1,3,"Scene: Central Perk, everyone but Phoebe is th...",8.0
3,1,4,"Scene: Central Perk, everyone is there except ...",8.0
4,1,5,"Scene: Central Perk, all six are there.]\nMoni...",8.4


In [7]:
X_raw = df.Transcript.values

In [25]:
def bag_of_words(docs):
    stop_words = nltk.corpus.stopwords.words('english')
    bow = set()
    for doc in docs:
        doc = doc.replace('\n',' ')
        doc = doc.strip()
        doc = doc.translate(str.maketrans('','',string.punctuation))
        #print(doc)
        text = doc.split(" ")
        for word in text:
            if word not in stop_words and word not in string.digits:
                bow.add(word)
    return bow

In [26]:
bow = bag_of_words(X_raw)

In [27]:
from tqdm import tqdm
X_train = pd.DataFrame(index = range(X_raw.shape[0]),columns = list(bow))
for idx, doc in tqdm(enumerate(X_raw)):
    X_train.iloc[idx] = np.zeros(len(bow))
    doc = doc.replace('\n',' ')
    doc = doc.strip()
    doc = doc.translate(str.maketrans('','',string.punctuation))
    text = doc.split(" ")
    for word in text:
        if word in bow:
            X_train[word][idx] +=1

227it [00:44,  5.10it/s]


In [28]:
X_train

,eggroll,Holyfield,classifieds,22,hair—Wow,Hersheys,purple,street,Leon,robbery,...,climbs,gi,sconces,alongside,�s,propane,Wax,Thread,spokes,r
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
y_train = df.Rating.values

In [33]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(verbose =1,n_jobs=-1)
rfr.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.3s finished


RandomForestRegressor(n_jobs=-1, verbose=1)

In [35]:
rfr.score(X_train,y_train)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


0.8834785235015061

In [51]:
test_raw = pd.read_csv('friends_test_file.csv', usecols=['Transcripts'])
test_raw.head()

,Transcripts
0,\n\nThe One Where Emma Cries\n\n\n\nThe One Wh...
1,\n\n\n\n903 - The One With The Pediatrician\n\...
2,\n\n\nUntitled Document\n\n\n\n906 - The One W...
3,\n\n\nThe One With Ross's Inappropriate Song\n...
4,\n\n\n908 - The One With Rachel's Other Sister...


In [52]:
test_raw = test_raw.Transcripts.values

In [53]:
X_test = pd.DataFrame(index = range(test_raw.shape[0]),columns = list(bow))
for idx, doc in tqdm(enumerate(test_raw)):
    X_test.iloc[idx] = np.zeros(len(bow))
    doc = doc.replace('\n',' ')
    doc = doc.strip()
    doc = doc.translate(str.maketrans('','',string.punctuation))
    text = doc.split(" ")
    for word in text:
        if word in bow:
            X_test[word][idx] +=1

35it [00:07,  4.77it/s]


In [54]:
X_test

,eggroll,Holyfield,classifieds,22,hair—Wow,Hersheys,purple,street,Leon,robbery,...,climbs,gi,sconces,alongside,�s,propane,Wax,Thread,spokes,r
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
preds = rfr.predict(X_test)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


In [63]:
predictions = pd.DataFrame(preds, columns = ['ratings'])

In [67]:
predictions['ID'] = predictions.index+1

In [69]:
predictions.to_csv('bow_predictions.csv',index=False) #Score of 0.17373